Un modelo lineal es uno de los modelos más simples y más ampliamente usados para hacer predicciones. La predicción se obtiene como una combinación lineal de las *features* de los datos ejemplo de entrada.

Los modelos lineales se usan principalmente para regresión aunque también se pueden usar para hacer clasificación.

## Modelo lineal para regresión

La fórmula general para regresión seria algo así:

$$ \hat{y} = w[0] * x[0] + w[1] * x[1] + ... + w[p] * x[p] + b $$

$ x[0] $ a $ x[p] $ son las *features* (en este caso de *p* dimensiones) para un único punto. $w$ y $b$ son los parámetros del modelo que se aprenden a partir de los datos. $\hat{y}$ es la predicción que hace el modelo.

En el caso de una única dimension lo anterior se reduce a:

$$ \hat{y} = w[0] * x[0] + b $$

Que os debería de sonar de algo que aplicamos muchas veces en muchos campos. $w[0]$ es la pendiente y $b$ es la ordenada al origen, el *offset* o término independiente.

Otra forma de ver lo anterior sería pensar en ello como que $w$ son los pesos para cada *feature* y la predicción no es más que la suma pesada de las *features*.

Los modelos lineales para regresión son modelos de regresión para los cuales la predicción viene dada por una línea en datos de una dimensión, un plano cuando tenemos datos de dos dimensiones oa hiperplano para datos de más dimensiones.

Existen muchos modelos lineales para regresión. La principal diferencia entre ellos se encuentra en la forma de calcular $w$ y $b$ y en la forma de controlar la complejidad del modelo.

## Regresión lineal

También conocido como ajuste por mínimos cuadrados u *Ordinary Least Squares* (OLS). Es el más simple y más clásico modelo lineal para regresión.

La regresión lineal aprende los parámetros $w$ y $b$ minimizando el error cuadrático medio (MSE, por sus siglas en inglés, *Mean Square Error*) obtenido entre las predicciones y los valores objetivo reales.

* La regresión lineal no tiene parámetros, lo cual es un beneficio, pero ello lleva a no tener forma de controlar la complejidad del modelo.

Un ejemplo en Python puro sería algo como lo siguiente:

In [ ]:
# https://en.wikipedia.org/wiki/Correlation_and_dependence#Pearson's_product-moment_coefficient
# https://en.wikipedia.org/wiki/Simple_linear_regression#Fitting_the_regression_line

from math import sqrt

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lib.datasets import make_wave

class RegresionLineal:
    
    def __init__(self):
        self.X = None
        self.y = None
        self.score_ = None
        self.intercept_ = None
        self.score2_ = None
        self.intercept2_ = None
        
    def fit(self, x, y):
        xy = 0
        x2 = 0
        xm = self._mean(x)
        ym = self._mean(y)
        m = len(y)
        for xi, yi in zip(x, y):
            xy += xi * yi
            x2 += xi ** 2
        self.coef_ = (xy - m * xm * ym) / (x2 - m * xm ** 2)
        self.intercept_ = ym - self.coef_ * xm
    
    def fit2(self, x, y, w0=0, b0=0, alpha=0.001, iters=100):
        m = len(y)
        for i in range(iters):
            b_grad = 0
            w_grad = 0
            for j in range(m):
                b_grad += (2/m) * (w0 * x[j] + b0 - y[j])
                w_grad += (2/m) * x[j] * (w0 * x[j] + b0 - y[j])
            b0 = b0 - (alpha * b_grad)
            w0 = w0 - (alpha * w_grad)
        self.score2_ = w0
        self.intercept2_ = b0
        
    def _mean(self, x):
        return sum(x) / len(x)
    
    def _std(self, x):
        res = sqrt(sum((xi - self._mean(x))**2 for xi in x) / (len(x) - 1))
        return res
    
    def score(self, x, y):
        u = 0
        v = 0
        ym = self._mean(y)
        for xi, yi in zip(x, y):
            u += (yi - (self.coef_ * xi + self.intercept_)) ** 2
            v += (yi - ym) ** 2
        return 1 - u / v

Vamos a intentar usarlo a ver lo que obtenemos. Primero obtenemos unos datos con los que trabajar

In [ ]:
X, y = make_wave()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

fig, ax = plt.subplots()
ax.scatter(X_train, y_train, c='k')
ax.scatter(X_test, y_test, c='b')

Instanciamos nuestro modelo

In [ ]:
model = RegresionLineal()

Lo entrenamos con los datos de entrenamientos:

In [ ]:
model.fit(X_train, y_train)

Vamos a ver la pendiente que obtenemos:

In [ ]:
model.coef_

Y el término independiente

In [ ]:
model.intercept_

¿Cómo ajusta el modelo? ($ R^2 $)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_test, y_test) #### ¿¿¿¿????

In [ ]:
import numpy as np

np.corrcoef(X_test.T, y_test) ** 2 # ¿¿¿¿????¿¿¿¿????¿¿¿¿????

Esto lo comentamos en un segundo... ----> Gradient descent

In [ ]:
model.fit2(X_train, y_train, iters=100) # probar valores más altos de iters

In [ ]:
model.score2_, model.intercept2_

Vamos a hacerlo ahora con `scikit-learn`.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lib.datasets import make_wave

In [ ]:
X, y = make_wave()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
linreg = LinearRegression()

In [ ]:
linreg.fit(X_train, y_train)

In [ ]:
print(f"{linreg.coef_} * x + {linreg.intercept_}")

In [ ]:
print(linreg.score(X_train, y_train))

In [ ]:
print(linreg.score(X_test, y_test))

In [ ]:
print(dir(linreg))

## [INCISO] Gradient descent

El gradiente descendiente se usa para optimización. Se usa mucho en aprendizaje automático porque es sencillo de entender. No tiene mucho sentido usarlo en un modelo de regresión lineal ya que tiene una solución directa pero se usa en otros algoritmos.

Lo que hacemos es básicamente la optimización de una función de pérdida para encontrar nuestros parámetros. En el caso de la regresión lineal lo que se optimiza (minimiza en este caso) es el error y será algo así:

$$ f(w, b) = \frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

Lo que hacemos ahora es obtener el gradiente:

$$ \frac{\partial{f(w, b)}}{\partial{w}} = \frac{-2}{N}\sum_{i=1}^{N}x_i*(y_i - (w*x_i+b))^2 $$
$$ \frac{\partial{f(w, b)}}{\partial{b}} = \frac{-2}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

Los parámetros los vamos actualizando de forma iterativa usando una tasa de aprendizaje (*learning rate*):

$$ w_{i+1} = w_{i} - (\alpha * \frac{\partial{f(w, b)}}{\partial{w}}) $$
$$ b_{i+1} = b_{i} - (\alpha * \frac{\partial{f(w, b)}}{\partial{b}}) $$

Podemos empezar con unos valores aleatorios para $ w_0 $ y $b_0$. Podemos iterar muchas veces o poner un valor de error donde nuestros parámetros cambian menos que ese valor de error y consideramos que ya es suficientemente correcto.

En `scikit-learn`, si queremos hacer una regresión lineal usando *Gradiend descent* podríamos usar lo siguiente:

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
linreg_sgd = SGDRegressor()
linreg_sgd.fit(X_train, y_train)

In [ ]:
print(linreg_sgd.coef_, linreg_sgd.intercept_, linreg_sgd.score(X_train, y_train))

In [ ]:
help(SGDRegressor)

## Ejemplo de clasificación

Para una clasificación, por ejemplo, binaria funciona de la siguiente forma, la fórmula es muy similar a la que hemos visto para la regresión lineal pero en lugar de devolver la suma pesada de las *features* lo que hacemos es poner un umbral en 0 y si el valor predicho es inferior a 0 entonces estamos prediciendo la clase -1 y si es superior a 0 entonces estamos prediciendo la clase +1. Esta regla de predicción es común para todos los modelos lineales para clasificación. Como en los modelos lineales de regresión tambien encontramos diferentes formas de encontrar las coeficientes $w$ y el término independiente $b$.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X, y = make_blobs(n_samples=50, centers=2, random_state=0, cluster_std=0.60)

In [ ]:
fig, ax = plt.subplots()

ax.plot(X[:, 0], X[:,1], 'o')

In [ ]:
clf1 = SGDRegressor(loss="squared_loss", alpha=0.01, max_iter=200, fit_intercept=True, penalty=None)
clf2 = SGDRegressor(loss="squared_epsilon_insensitive", alpha=0.01, max_iter=200, fit_intercept=True)

In [ ]:
clf1.fit(X, y)
clf2.fit(X, y)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
idx = y == 0
ax1.scatter(X[idx,0], X[idx,1], c='b')
ax2.scatter(X[idx,0], X[idx,1], c='b')
ax1.scatter(X[~idx,0], X[~idx,1], c='g')
ax2.scatter(X[~idx,0], X[~idx,1], c='g')

for x1 in np.arange(0,4, 0.1):
    for x2 in np.arange(0,6, 0.1):
        clase = clf1.predict([[x1, x2]])[0]
        color = "b" if clase <= 0.5 else "g"
        ax1.scatter(x1, x2, c=color, marker="*", s=50, alpha=0.5)
        clase = clf2.predict([[x1, x2]])[0]
        color = "b" if clase <= 0.5 else "g"
        ax2.scatter(x1, x2, c=color, marker="*", s=50, alpha=0.5)